In [1]:
from copy import deepcopy
import os
import math
import shutil
import re
from glob import glob
from collections import defaultdict, Counter

import numpy as np
from tqdm import tqdm
import pandas as pd
pd.set_option('display.max_colwidth', None)

import rispy
mapping = deepcopy(rispy.TAG_KEY_MAPPING)
mapping['AN'] = 'extra'
mapping['AB'] = 'orig_abstract'
mapping['CN'] = 'call_number'
mapping['SV'] = 'series_volume'

In [2]:
dump_dir = '../data/bntl-db-csv-dump-4March2024'

### Recensies

*Add to reviews which publication they review (under "RI" = reviewed item).*

We first load the original data table for all publications in the original dump:

In [3]:
publications = pd.read_csv(f"{dump_dir}/publications.csv", header=0, parse_dates=['creation_date', 'modification_date'])
publications['id'] = publications['id'].fillna('')
publications.sample(3)

,id,ppn,type,jaar,title_description,abstract,url,containing_publication_id,containing_publication_ppn,page_numbers,creator,creation_date,modifier,modification_date,canonical_url
85982,130075,242506054,artikeltijdschrift,2002,"Op wie lijkt meneer Taats? / [door] A.H. den Boef.\nIn: Ons erfdeel: vol. 45 (2002), afl. 4 (sep-okt), pag. 591-593.","Over: Vogelaar, Jacq. Taats onder mannen. Amsterdam, 2001.",NaN,158504.0,831080663,"vol. 45 (2002), afl. 4 (sep-okt), p. 591-593.",1,NaT,NaN,NaT,NaN
75938,108236,183918649,artikeltijdschrift,1958,"Souvenir, souvenir, que me veux-tu? / J.W.F. Werumeus Buning.\nIn: A. Roland Holst; [onder red. van Bert Bakker, W.Gs Hellinga, Ed Hoornik en Bert Voeten]. Speciaal nr. van: Maatstaf ('s-Gravenhage) vol. 6 (1958-1959), afl. 2-3 (mei-juni 1958), pag. 182-185.","Ook in: Gids (Amst.): 121 (1958) I, 5-6 (mei-juni) 385-388; en in: A. Roland Holst zeventig jaar; onder red. van Bert Bakker, W.Gs Hellinga, Ed. Hoornik en Bert Voeten. Den Haag [etc., 1958], p. 101-104.",NaN,159166.0,831706228,"vol. 6 (1958-1959), afl. 2-3 (mei-juni), p. 182-185.",1,NaT,NaN,NaT,NaN
23393,198308,862656680,artikelboek,1977,"Bousset, Hugo. J. M. A. Biesheuvel - Eva versus Moby God. \r\nIn: Bousset, Hugo. Woord en schroom: enige trends in de Nederlandse prozaliteratuur 1973-1976. Nijmegen: Gottmer [etc.]; Brugge: Orion, 1977, p. 125-129.\r\n(Orions literair atelier).","Over: Biesheuvel, J. M. A. Slechte mensen. Amsterdam: De Harmonie, 1973.",NaN,NaN,NaN,p. 125-129.,1,NaT,19.0,2015-05-04 10:01:51,j_m_a_biesheuvel_-_eva_versus/bousset_hugo


We are going to extract title descriptions for including that in the RIS dump, so we much sure that these are clean:

In [4]:
publications['title_description'] = [' '.join(p.split()).strip() for p in publications['title_description']]

We extract a review's title description and check to which (source) id this review was mapped:

In [5]:
#publications[publications['title_description'] == 'Kestemont, Mike. Eind goed, al goed? In: Queeste: 18 (2011) 1, 81-84.']['id']
publications[publications['title_description'] == 'Een waaier van emoties. Nijmegen: Vantilt. 23 p. Speciaal nummer van: Filter (Nijmegen): 25 (2018) 4 (dec) 3-26.']


,id,ppn,type,jaar,title_description,abstract,url,containing_publication_id,containing_publication_ppn,page_numbers,creator,creation_date,modifier,modification_date,canonical_url
2973,314016,NaN,specialetijdschriftaflevering,2018,Een waaier van emoties. Nijmegen: Vantilt. 23 p. Speciaal nummer van: Filter (Nijmegen): 25 (2018) 4 (dec) 3-26.,"P. 3 Inleiding, door de redactie; p. 26 Noten; p. 26 Bibliografie ""Koudwatervrees"".",NaN,NaN,NaN,NaN,451,2023-12-06 09:35:47,451.0,2023-12-06 10:03:06,een_waaier_van_emoties_nijmege/n_v_t


Next, we look up the (target) ID of the publication that was reviewed:

In [6]:
print(publications[publications['title_description'] == "Draux, Roland. Beatrijs' biecht: stilistisch en semiotisch onderzoek. Köln: Lambert Academic Publishing, 2009. 253 p."]['id'])

181127    273488
Name: id, dtype: int64


We now create three lookup dicts:
- `title2id`: to map a title description to a publication ID;
- `title2ptype`: to map a publication's title descirption to its publication type;
- `id2title`: to map a publication ID to a title description.

In [7]:
title2id = dict(zip(publications['title_description'], publications['id']))
id2title = dict(zip(publications['id'], publications['title_description']))

In [8]:
title2ptypes = defaultdict(set)
for td, pt in zip(publications['title_description'], publications['type']):
    title2ptypes[td].add(pt)

We now create a linkage dict (`review2reference`), that maps the IDs of reviews, to the IDs of the publication that they review:

In [9]:
references = pd.read_csv(f'{dump_dir}/publication_publications.csv')
reference2publication = dict(zip(references['publication_id'], references['referenced_publication_id']))

Now, we inverse our lookup dictionary, to that books can be mapped to a list of their reviews:

In [10]:
publication2references = defaultdict(list)
for review_id, book_id in reference2publication.items():
    publication2references[book_id].append(review_id)

def get_reviews(pub_title):
    try:
        pub_id = title2id[pub_title]
    except KeyError:
        return None
    
    reviews = []
    for potential_review_id in publication2references[pub_id]:
        potential_review_title = id2title[potential_review_id]
        if 'recensie' in title2ptypes[potential_review_title]:
            try:
                reviews.append(potential_review_title)
            except KeyError:
                continue
    
    if reviews:
        return ' | '.join(reviews)
    

def get_chapters(pub_title):
    try:
        pub_id = title2id[pub_title]
    except KeyError:
        return None
    
    chapters = []
    for potential_chapter_id in publication2references[pub_id]:
        potential_chapter_title = id2title[potential_chapter_id]
        if 'artikelboek' in title2ptypes[potential_chapter_title]:
            try:
                chapters.append(potential_chapter_title)
            except KeyError:
                continue
    
    if chapters:
        return ' | '.join(chapters)
    
def get_articles(pub_title):
    try:
        pub_id = title2id[pub_title]
    except KeyError:
        return None
    
    articles = []
    for potential_article_id in publication2references[pub_id]:
        potential_article_title = id2title[potential_article_id]
        if 'artikeltijdschrift' in title2ptypes[potential_article_title]:
            try:
                articles.append(potential_article_title)
            except KeyError:
                continue
    
    if articles:
        return ' | '.join(articles)


In [11]:
review_id = title2id["Kestemont, Mike. Eind goed, al goed? In: Queeste: 18 (2011) 1, 81-84."]
print(review_id)
book_id = reference2publication[review_id]
print(book_id)
book_title = id2title[book_id]
print(book_title)

274189
273488
Draux, Roland. Beatrijs' biecht: stilistisch en semiotisch onderzoek. Köln: Lambert Academic Publishing, 2009. 253 p.


In [12]:
t = "Ene andre tale: tendensen in de Middelnederlandse late ridderepiek; onder redactie van An Faems en Marjolein Hogenbirk. Hilversum: Verloren, 2012. 318 p. (Middeleeuwse studies en bronnen; 131)."
print(get_reviews(t))
print(get_chapters(t))

Besamusca, Bart. De (late) Middelnederlandse ridderepiek belicht. In: Queeste (Hilversum): 20 (2013) 1, 45-49. | Bouwmeester, Gerard. Nieuwe inzichten in de late ridderepiek. In: Spiegel der letteren (Leuven): 55 (2013) 4, 537-539.
Janssens, Jef. De Middelnederlandse ridderepiek in de veertiende eeuw: postmodern of voorzichtig vernieuwend? In: Ene andre tale: tendensen in de Middelnederlandse late ridderepiek; onder redactie van An Faems en Marjolein Hogenbirk. Hilversum: Verloren, 2012, p. 37-52. (Middeleeuwse studies en bronnen; 131). | Meulen, Janet F. van der. Vrouwen van Avesnes: een nieuwe Alexander in de Lage Landen. In: Ene andre tale: tendensen in de Middelnederlandse late ridderepiek; onder redactie van An Faems en Marjolein Hogenbirk. Hilversum: Verloren, 2012, p. 55-81. (Middeleeuwse studies en bronnen; 131). | Reynders, Anne. "Ghi heren, ic houde in ware wort dat ghi van Alexandre gehort hebt": de Middelnederlandse vertalingen van de Oudfranse "Florimont" en "Voeux du paon

In [13]:
llm_path = '../data/llm-dump'

for decade_folder in sorted(glob(f'{llm_path}/*')):
    print(':::', decade_folder, ':::')
    decade = decade_folder.split('/')[-1]
    
    for ris_path in sorted(glob(f'{decade_folder}/*_consolidated.ris')):
        print(ris_path)
        with open(ris_path) as f:
            entries = rispy.load(f, encoding='utf-8', mapping=mapping)
        
        for idx, entry in tqdm(list(enumerate(entries))):
            if 'extra' in entry:
                pub_title = ' '.join(entry['extra'].split()).strip()
            else:
                # this means that we weren't able to structure the item via the LLM...
                pub_title = ' '.join(entry['title'].split()).strip()
            
            # add to reviews which book they review:
            if 'keywords' in entry and 'recensie' in entry['keywords']:
                ref_title = None
                try:
                    pub_id = title2id[pub_title]
                    ref_id = reference2publication[pub_id]
                    ref_title = id2title[ref_id]
                except KeyError:
                    pass

                if ref_title:
                    entry['short_title'] = ref_title

                if not 'title' in entry or entry['title'].strip() in ('titel', 'title'):
                    entry['title'] = '[Zonder titel]'
                
                entries[idx] = entry
            
            reviews = get_reviews(pub_title)
            if reviews:
                if entry['type_of_reference'] != 'WEB':
                    entry['call_number'] = reviews
                else:
                    entry['language'] = reviews
                entries[idx] = entry
            
            if 'boek' in title2ptypes[pub_title]:
                chapters = get_chapters(pub_title)
                if chapters:
                    if 'extra' in entry:
                        entry['extra'] += "\nHOOFDSTUKKEN: " + chapters
                    else:
                        entry['extra'] = "HOOFDSTUKKEN: " + chapters
                    entries[idx] = entry

            if 'specialetijdschriftaflevering' in title2ptypes[pub_title]:
                articles = get_articles(pub_title)
                if articles:
                    if 'extra' in entry:
                        entry['extra'] += "\nARTIKELS: " + articles
                    else:
                        entry['extra'] = "ARTIKELS: " + articles
                    entries[idx] = entry

            if entry['type_of_reference'] == 'CHAP':
                try:
                    pub_id = title2id[pub_title]
                    containing_title = None
                
                    containing_id = reference2publication[pub_id]
                    containing_title = id2title[containing_id]
                except KeyError:
                    continue

                if containing_title:
                    if 'extra' in entry:
                        entry['extra'] += "\nBOEK: " + containing_title
                    else:
                        entry['extra'] = "\nBOEK: " + containing_title
                    entries[idx] = entry

            if entry['type_of_reference'] == 'JOUR' and 'extra' in entry:
                if 'speciaal' in entry['extra'].lower():
                    if 'keywords' in entry:
                        entry['keywords'].append('Bijdrage speciaal tijdschriftnummer')
                    else:
                        entry['keywords'] = ['Bijdrage speciaal tijdschriftnummer']

        out_ris_path = ris_path.replace('.ris', f'_link_{decade}.ris')
        with open(out_ris_path, 'w') as bibliography_file:
            rispy.dump(entries, bibliography_file, mapping=mapping)

::: ../data/llm-dump/1940s :::
../data/llm-dump/1940s/BOOK_consolidated.ris


100%|██████████| 1442/1442 [00:00<00:00, 131425.17it/s]


../data/llm-dump/1940s/CHAP_consolidated.ris


100%|██████████| 1764/1764 [00:00<00:00, 237642.20it/s]


../data/llm-dump/1940s/JFULL_consolidated.ris


...


100%|██████████| 6051/6051 [00:00<00:00, 238276.03it/s]


../data/llm-dump/misc/JOUR_consolidated.ris


100%|██████████| 3483/3483 [00:00<00:00, 294442.42it/s]


../data/llm-dump/misc/WEB_consolidated.ris


100%|██████████| 32/32 [00:00<00:00, 139664.65it/s]
